In [1]:
import os

# 1. Setup Paths

In [21]:
CUSTOM_MODEL_NAME = 'tomato_detection_v2_640x640' 
PRETRAINED_MODEL_NAME = 'tomato_detection_v1_640x640'
PRETRAINED_MODEL_URL = 'https://github.com/JannesNebendahl/TomatoCounter/raw/main/pretrained_models/tomato_detection_v1_640x640.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'
DATASET_NAME = 'dataset.tar.gz'
TRAINSET_NAME = 'trainset.record'
TESTSET_NAME = 'testset.record'


paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

file_paths = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME),
    'DATASET': os.path.join(paths['IMAGE_PATH'], DATASET_NAME)
}

In [22]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

# 2. Download pretrained Tensorflow Model and install TFOD

In [4]:
if os.name=='nt':
    !pip install wget
    import wget

In [5]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    command = "git clone https://github.com/tensorflow/models {}".format(paths['APIMODEL_PATH'])
    print(command)
    !{command}

Install TFOD (TensorFlow Object Detection)

In [6]:
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 

        1 file(s) moved.
        1 file(s) copied.
running build
running build_py
copying object_detection\protos\anchor_generator_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\argmax_matcher_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\bipartite_matcher_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\box_coder_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\box_predictor_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\calibration_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\center_net_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\eval_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\faster_rcnn_box_coder_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\faster_rcnn_pb2.py -> build\lib\object_detection\protos
copying object_detecti

c:\Users\JannesPC\anaconda3\envs\TomatoCounter\lib\site-packages\setuptools\command\install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
c:\Users\JannesPC\anaconda3\envs\TomatoCounter\lib\site-packages\setuptools\command\easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
zip_safe flag not set; analyzing archive contents...
object_detection.core.__pycache__.densepose_ops.cpython-310: module references __file__
object_detection.core.__pycache__.preprocessor.cpython-310: module MAY be using inspect.stack
object_detection.utils.__pycache__.autoaugment_utils.cpython-310: module MAY be using inspect.stack
error: numpy 1.24.0 is installed but numpy<1.23.0,>=1.14.3 is required by {'apache-beam'}


Obtaining file:///C:/Users/JannesPC/Documents/AiProjects/TomatoCounter/Tensorflow/models/research/slim
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: slim
    Found existing installation: slim 0.1
    Uninstalling slim-0.1:
      Successfully uninstalled slim-0.1
  Running setup.py develop for slim


Verify that everything is setup correctly.  
The script should execute with 'OK (skipped=1)'

In [23]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

2022-12-29 16:58:28.960706: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-12-29 16:58:28.961545: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-29 16:58:38.522176: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-12-29 16:58:38.522954: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-29 16:58:38.527778: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: Yoga-7
2022-12-29 16:58:38.528000: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: Yoga-7
Running tests under Python 3.10.8: c:\Users\JannesPC\anaconda3\envs\TomatoCounter\python.exe
[ RUN      ] ModelBuilderTF2Tes

In [8]:
import object_detection

In [9]:
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

        1 file(s) moved.


x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.index
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/saved_model.pb
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.index


# 3. Create Label Map

In [24]:
labels = [{'name':'RipeTomato', 'id':1}, {'name':'UnripeTomato', 'id':2}]

with open(file_paths['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 4. Upload dataset to Colab

Upload the dataset to google drive

In [ ]:
if os.name =='posix':
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
GOOGLE_DRIVE_PATH = '/content/drive/MyDrive/TomatoCounter'

if os.name =='posix':
    from google.colab import drive

    drive.mount('/content/drive')

    import shutil
    shutil.copyfile(os.path.join(GOOGLE_DRIVE_PATH, DATASET_NAME), os.path.join(paths['IMAGE_PATH'], DATASET_NAME))

Now your google drive is connected and you have to copy the dataset into the image directory

Verify that dataset is available

In [25]:
if not os.path.isfile(file_paths['DATASET']):
  print('No dataset available')
else:
  print('Dataset available')

Dataset available


# 5. Create TF records

In [26]:
if os.path.exists(file_paths['DATASET']):
  !tar -zxvf {file_paths['DATASET']}

x Tensorflow/workspace/images/trainset/
x Tensorflow/workspace/images/trainset/DSC_0010.JPG
x Tensorflow/workspace/images/trainset/DSC_0010.xml
x Tensorflow/workspace/images/trainset/DSC_0011.JPG
x Tensorflow/workspace/images/trainset/DSC_0011.xml
x Tensorflow/workspace/images/trainset/DSC_0012.JPG
x Tensorflow/workspace/images/trainset/DSC_0012.xml
x Tensorflow/workspace/images/trainset/DSC_0013.JPG
x Tensorflow/workspace/images/trainset/DSC_0013.xml
x Tensorflow/workspace/images/trainset/DSC_0014.JPG
x Tensorflow/workspace/images/trainset/DSC_0014.xml
x Tensorflow/workspace/images/trainset/DSC_0015.JPG
x Tensorflow/workspace/images/trainset/DSC_0015.xml
x Tensorflow/workspace/images/trainset/DSC_0016.JPG
x Tensorflow/workspace/images/trainset/DSC_0016.xml
x Tensorflow/workspace/images/trainset/DSC_0017.JPG
x Tensorflow/workspace/images/trainset/DSC_0017.xml
x Tensorflow/workspace/images/trainset/DSC_0018.JPG
x Tensorflow/workspace/images/trainset/DSC_0018.xml
x Tensorflow/workspace/i

In [27]:
if not os.path.exists(file_paths['TF_RECORD_SCRIPT']):
    import urllib.request

    destination = os.path.join(paths['SCRIPTS_PATH'], 'generate_tfrecord.py')
    urllib.request.urlretrieve("https://raw.githubusercontent.com/JannesNebendahl/TomatoCounter/main/scripts/generate_tfrecord.py",destination)

In [36]:
!python {file_paths['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'trainset')} -l {file_paths['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], TRAINSET_NAME)} 
!python {file_paths['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'testset')} -l {file_paths['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], TESTSET_NAME)}

Successfully created the TFRecord file: Tensorflow\workspace\annotations\trainset.record
Successfully created the TFRecord file: Tensorflow\workspace\annotations\testset.record
Successfully created the CSV file: csv.csv


# 6. Copy Model Config to Training Folder

In [48]:
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

        1 file(s) copied.


# 7. Update Config for Transfer Learning

In [49]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [50]:
config = config_util.get_configs_from_pipeline_file(file_paths['PIPELINE_CONFIG'])
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matc

In [51]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(file_paths['PIPELINE_CONFIG'], "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)  

In [52]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-21')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= file_paths['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], TRAINSET_NAME)]
pipeline_config.eval_input_reader[0].label_map_path = file_paths['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], TESTSET_NAME)]

In [53]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(file_paths['PIPELINE_CONFIG'], "wb") as f:
    f.write(config_text)  

# 8. Train the model

In [54]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],file_paths['PIPELINE_CONFIG'])
print(command)

python Tensorflow\models\research\object_detection\model_main_tf2.py --model_dir=Tensorflow\workspace\models\tomatoDetectionV1 --pipeline_config_path=Tensorflow\workspace\models\tomatoDetectionV1\pipeline.config --num_train_steps=2000


In [82]:
!{command}

^C


2022-12-28 22:32:30.925507: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-12-28 22:32:30.926454: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-28 22:32:36.851978: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-12-28 22:32:36.852175: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-28 22:32:36.854457: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: Yoga-7
2022-12-28 22:32:36.854707: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: Yoga-7
2022-12-28 22:32:36.860921: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with on

# 9. Freezing the Graph

In [ ]:
FREEZE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'exporter_main_v2.py ')
command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(FREEZE_SCRIPT ,file_paths['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['OUTPUT_PATH'])
print(command)

In [ ]:
!{command}

# 10. Zip and Store in Google Drive

In [ ]:
!zip -r {'/content/drive/MyDrive/TomatoCounter/' + CUSTOM_MODEL_NAME + '.zip'} {paths['CHECKPOINT_PATH']}